In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Reshape the input data to be of shape (batch_size, timesteps, features)
x_train = x_train.reshape(-1, 28, 28).astype('float32')  # (60000, 28, 28)
x_test = x_test.reshape(-1, 28, 28).astype('float32')    # (10000, 28, 28)

# Normalize pixel values to be between 0 and 1
x_train /= 255.0
x_test /= 255.0

# Define a custom model class using subclassing
class MyModel(keras.Model):
    def __init__(self, hidden_units):
        super(MyModel, self).__init__()
        # Define layers in the constructor
        self.bidirectional_rnn_1 = layers.Bidirectional(layers.SimpleRNN(hidden_units, return_sequences=True, activation="relu"))
        self.bidirectional_rnn_2 = layers.Bidirectional(layers.SimpleRNN(hidden_units, activation="relu"))
        self.dense = layers.Dense(10)  # Output layer (10 classes for MNIST)

    def call(self, inputs):
        # Define the forward pass in the call method
        x = self.bidirectional_rnn_1(inputs)
        x = self.bidirectional_rnn_2(x)
        return self.dense(x)

# Create an instance of the model
model = MyModel(512)

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=1, verbose=2)

# Save the model to HDF5 format
model.save('my_model.h5')  # This saves the model to 'my_model.h5'

# Alternatively, save the model in SavedModel format
# model.save('my_model_saved')  # This saves in SavedModel format

# Now let's load the model
loaded_model = keras.models.load_model('my_model.h5')  # Load from HDF5

# Alternatively, load from SavedModel format
# loaded_model = keras.models.load_model('my_model_saved')  # Load from SavedModel

# Evaluate the loaded model
loaded_model.evaluate(x_test, y_test, verbose=2)

# You can continue training the loaded model
# loaded_model.fit(x_train, y_train, batch_size=32, epochs=1, verbose=2)
